In [1]:
import unicodecsv                               # csv reader
import re                                       # regular expressions
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
import csv
import pandas
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from collections import Counter
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
nltk.download('averaged_perceptron_tagger')


# To do preprocessing
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
import numpy as np # To compute the average results

from random import shuffle # To shuffle the dataset


# To use feature selection in the Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#this is the function for preprocessing the data, was taken from lab2

def preProcess(text):
    # should return a list of tokens
    
    # word tokenisation, including punctuation removal
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # lowercasing
    tokens = [t.lower() for t in tokens]

    # stopword removal- benefits are it removes rare words, though bad for bigram relations
    if False:
        stop = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop]
    
    # lemmatisation
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t] # ensure no empty space
    
    
    return tokens

In [3]:
#Loading the training set and droping null columns
Corpus = pd.read_csv(r'training.csv',encoding='latin-1',names=['text', 'character', 'gender'])
Corpus = Corpus.dropna()
print(Corpus)
#Loading the test set and droping null columns
Test = pd.read_csv(r'test.csv',encoding='latin-1',names=['text', 'character', 'gender'])
Test = Test.dropna()

#preprocessing the data with the preProcess function
for index, line in Corpus.iterrows():
    Corpus['text'][index] = preProcess(line.text)


#preprocessing the data with the preProcess function
for index, line in Test.iterrows():
    Test['text'][index] = preProcess(line.text)
  


                                                    text  character  gender
0                                       Someone had fun.       SEAN    male
1      It's no problem, honestly. Go on, go and open ...    SHIRLEY  female
2      Last night was better than ever. What's all th...        MAX    male
3          Have you checked the answerphone?  Any calls?        IAN    male
4                                        Oscar's asleep.        MAX    male
5                             Dead Arm? What's Dead Arm?      MINTY    male
6                                                 Rox...     RONNIE  female
7      Oh no, I wouldn't do that. Here's the menu. Sh...        IAN    male
8              I think I've got someone in line already.     RONNIE  female
9                       Like drinking yourself to death?       SEAN    male
10     You trying to kiss me doesn't change anything....     STACEY  female
11     Well as long as the Queen likes it. That's the...        IAN    male
12     You k

In [4]:
#create a unigram on the training set
unigram = Counter()
for line in Corpus['text']:
    for w in line:
        unigram[w]+=1

#print(len(unigram),unigram)


#create a bigram
bigram = []
for tokens in Corpus['text']:
    for i in range(len(tokens)-1):
        bigram.append(tokens[i:i+2])

#postag
#a=[]
#b=[]
#for tokens in Corpus['text']:
#    tag = nltk.pos_tag(tokens)
#    b.append(tag)
    #print(tag)
#    grammar = "NP: {<DT>?<JJ>*<NN>}"
#    cp  =nltk.RegexpParser(grammar)
#    result = cp.parse(tag)
#    a.append(result)




In [5]:
import heapq
#we are using as feautures the 500 most common/frequent words from our vocabulary.
most_freq = heapq.nlargest(500, unigram, key=unigram.get)

#create bag of words for the Corpus data.
sentence_vectors = []
for text in Corpus['text']:
    i=len(text)
    sent_vec = []
    for token in most_freq:
        if token in text:
            sent_vec.append(1)
            #print(token,text)
        else:
            sent_vec.append(0)
    #sent_vec.append(i)
    #if you want to include in the classification the length of the sentance as an extra feature uncomment the above line.
    sentence_vectors.append(sent_vec)

   # print(sent_vec)
sentence_vectors = np.asarray(sentence_vectors)
#print(sentence_vectors)

#create bag of words for the Test data.
sentence_vectors_test = []
for text in Test['text']:
    e=len(text)
    sent_vec_test = []
    for token in most_freq:
        if token in text:
            sent_vec_test.append(1)
            #print(token,text)
        else:
            sent_vec_test.append(0)
    #sent_vec_test.append(e)
    # #if you want to include in the classification the length of the sentance as an extra feature uncomment the above line.
    sentence_vectors_test.append(sent_vec_test)

sentence_vectors_test = np.asarray(sentence_vectors_test)
sentence_vectors_test.shape



(1120, 500)

In [17]:
#IDF values
word_idf_values = {}
for token in most_freq:
    doc = 0
    for text in Corpus['text']:
        if token in text:
            doc += 1
    word_idf_values[token] = np.log(len(Corpus['text'])/(1 + doc))

#create the TF dictionary for each word    
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in Corpus['text']:
        doc_freq = 0
        for word in document:
            if token == word:
                  doc_freq += 1
        if len(document)==0:
            word_tf=0
        else:
            word_tf = doc_freq/len(document)
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector
    
#multiply IDF values with TF values    
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

tf_idf_model = np.asarray(tfidf_values)
tf_idf = tf_idf_model.transpose()
tf_idf.shape

(10071, 500)

In [7]:
#create the TF dictionary for each word for test  
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in Test['text']:
        doc_freq = 0
        for word in document:
            if token == word:
                  doc_freq += 1
        if len(document)==0:
            word_tf=0
        else:
            word_tf = doc_freq/len(document)
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector
    
#multiply IDF values with TF values    
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

tf_idf_model_test = np.asarray(tfidf_values)
tf_idf_test = tf_idf_model_test.transpose()


In [8]:
#split the training data to train and heldout with 70& and 30& respectively for the crossvalidation
Train_X, Heldout_X, Train_Y, Heldout_Y = model_selection.train_test_split(sentence_vectors,Corpus['gender'],test_size=0.3)
#By changing the X from sentence_vectors to tf_idf you will use as features the tf_idf method and not the bag of words
#If you want to classify on the character unocomment the below and comment the above
#Train_X, Heldout_X, Train_Y, Heldout_Y = model_selection.train_test_split(sentence_vectors,Corpus['character'],test_size=0.3)

Test_X = sentence_vectors_test # change to tf_idf_test to use as features the tf_idf method and not the bag of words
Test_Y = Test['gender']
FULL_X = sentence_vectors # change to tf_idf to use as features the tf_idf method and not the bag of words


In [9]:
#encode the label
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Heldout_Y = Encoder.fit_transform(Heldout_Y)
Test_Y = Encoder.fit_transform(Test_Y)
FULL_Y = Encoder.fit_transform(Corpus['gender'])
#FULL_Y = Encoder.fit_transform(Corpus['character']) if you want to classify on the character unocomment this and comment the line above

In [10]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(FULL_X,FULL_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X)

# Use accuracy_score function,recall_score,precision_score,f1_score to get the accuracy,recall,precision and f1 score
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM Recall Score -> ",recall_score(predictions_SVM, Test_Y,average='weighted')*100)
print("SVM Precision Score -> ",precision_score(predictions_SVM, Test_Y,average='weighted')*100)
print("SVM F1 Score -> ",f1_score(predictions_SVM, Test_Y,average='weighted')*100)

SVM Accuracy Score ->  59.017857142857146
SVM Recall Score ->  59.017857142857146
SVM Precision Score ->  59.09865946378551
SVM F1 Score ->  58.978985603617964


In [12]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVM, Heldout_X,Heldout_Y)
print(scores)

#the above is the crossvalidation function to check if our model is overfitting

C:\Users\spyro\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0.5218254  0.54220457 0.53326713]


In [13]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(FULL_X,FULL_Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(Naive, Heldout_X,Heldout_Y)
print(scores)

#the above is the crossvalidation function to check if our model is overfitting

[0.55257937 0.54716981 0.55412115]


C:\Users\spyro\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [15]:
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X)

# Use accuracy_score function,recall_score,precision_score,f1_score to get the accuracy,recall,precision and f1 score
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_Y,average='weighted')*100)
print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_Y,average='weighted')*100)
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y,average='weighted')*100)

Naive Bayes Accuracy Score ->  57.67857142857142
Naive Bayes Recall Score ->  57.67857142857142
Naive Bayes Precision Score ->  57.656962785114054
Naive Bayes F1 Score ->  57.64126131589502


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logistic_regression= LogisticRegression()
logistic_regression.fit(FULL_X,FULL_Y)
y_pred=logistic_regression.predict(Test_X)

# Use accuracy_score function,recall_score,precision_score,f1_score to get the accuracy,recall,precision and f1 score
print("Logistic Regression Accuracy Score -> ",accuracy_score(y_pred, Test_Y)*100)
print("Logistic Regression Recall Score -> ",recall_score(y_pred, Test_Y,average='weighted')*100)
print("Logistic Regression Precision Score -> ",precision_score(y_pred, Test_Y,average='weighted')*100)
print("Logistic Regression F1 Score -> ",f1_score(y_pred, Test_Y,average='weighted')*100)



C:\Users\spyro\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score ->  58.392857142857146
Logistic Regression Recall Score ->  58.392857142857146
Logistic Regression Precision Score ->  58.78241296518608
Logistic Regression F1 Score ->  58.38410042584361
